In [1]:
# --- MULTI-AGENT RESEARCH ASSISTANT: SALES INTELLIGENCE EDITION ---


%pip install -U langchain_openai langgraph
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("LANGSMITH_API_KEY")

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langchain-academy"

# --- Overview ---
# This combines Map-Reduce and Parallelization into one research assistant.
# It performs:
# 1️⃣ Map-Reduce → Identify the best-performing product in a category.
# 2️⃣ Parallelization → Generate quick summaries for each product.

from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.types import Send
from typing_extensions import TypedDict
from typing import Annotated
import operator
from pydantic import BaseModel
from IPython.display import Image

# --- Model ---
model = ChatOpenAI(model="gpt-4o", temperature=0)

# --- Prompts ---
products_prompt = """Generate a list of 4 products from this category: {category}."""
sales_summary_prompt = """Write a concise summary (2-3 lines) describing the sales trends, key markets, and growth opportunities for {product}."""
best_product_prompt = """Here are sales summaries of several products in the category {category}.

Choose the best-performing product overall and return its ID (starting from 0 for the first). Summaries:

{summaries}"""

# --- Models ---
class Products(BaseModel):
    products: list[str]

class ProductSummary(BaseModel):
    summary: str

class BestProduct(BaseModel):
    id: int

# --- State ---
class OverallState(TypedDict):
    category: str
    products: list
    summaries: Annotated[list, operator.add]
    best_product: str

# --- Node 1: Generate product list ---
def generate_products(state: OverallState):
    prompt = products_prompt.format(category=state["category"])
    response = model.with_structured_output(Products).invoke(prompt)
    return {"products": response.products}

# --- Node 2: Parallel summaries ---
class ProductState(TypedDict):
    product: str

def generate_summary(state: ProductState):
    prompt = sales_summary_prompt.format(product=state["product"])
    response = model.with_structured_output(ProductSummary).invoke(prompt)
    return {"summaries": [f"{state['product']}: {response.summary}"]}

def continue_to_summaries(state: OverallState):
    return [Send("generate_summary", {"product": p}) for p in state["products"]]

# --- Node 3: Reduce phase (find best product) ---
def best_product(state: OverallState):
    summaries = "\n\n".join(state["summaries"])
    prompt = best_product_prompt.format(category=state["category"], summaries=summaries)
    response = model.with_structured_output(BestProduct).invoke(prompt)
    return {"best_product": state["summaries"][response.id]}

# --- Node 4: Final reporting ---
def final_report(state: OverallState):
    print("\n=== FINAL SALES RESEARCH REPORT ===\n")
    for s in state["summaries"]:
        print(s)
    print(f"\n🏆 Best Performing Product:\n{state['best_product']}")
    print("\n✅ Research assistant analysis complete.")
    return {}

# --- GRAPH ---
graph = StateGraph(OverallState)
graph.add_node("generate_products", generate_products)
graph.add_node("generate_summary", generate_summary)
graph.add_node("best_product", best_product)
graph.add_node("final_report", final_report)

graph.add_edge(START, "generate_products")
graph.add_conditional_edges("generate_products", continue_to_summaries, ["generate_summary"])
graph.add_edge("generate_summary", "best_product")
graph.add_edge("best_product", "final_report")
graph.add_edge("final_report", END)

# --- Compile & Visualize ---
app = graph.compile()
Image(app.get_graph().draw_mermaid_png())

# --- RUN ---
for s in app.stream({"category": "Electronics"}):
    print(s)


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


{'generate_products': {'products': ['1. **Sony WH-1000XM4 Wireless Noise-Canceling Headphones**  \n   - Features: Industry-leading noise cancellation, up to 30 hours of battery life, touch sensor controls, and voice assistant compatibility.\n   - Price: $349.99\n   - Description: Experience the ultimate in wireless freedom with these premium headphones that deliver exceptional sound quality and comfort.\n\n2. **Apple iPad Air (5th Generation)**  \n   - Features: 10.9-inch Liquid Retina display, A14 Bionic chip, 64GB storage, and support for Apple Pencil (2nd generation).\n   - Price: $599.00\n   - Description: A powerful and versatile tablet that combines performance with portability, perfect for work, play, and creativity.\n\n3. **Samsung Galaxy S21 Ultra 5G Smartphone**  \n   - Features: 6.8-inch Dynamic AMOLED 2X display, 108MP camera, 5,000mAh battery, and 5G connectivity.\n   - Price: $1,199.99\n   - Description: A flagship smartphone that offers cutting-edge technology, stunning 

IndexError: list index out of range